# 0 - Imports

In [24]:
import pandas as pd
import inflection
import math

## 0.1. Helper functions

## 0.2. Loading data

In [11]:
#Leitura dos arquivos
df_sale_raw = pd.read_csv('Dataset\'s/train.csv', low_memory=False)
df_store_raw = pd.read_csv('Dataset\'s/store.csv', low_memory=False)


In [12]:
#Unindo os dois arquivos
df_raw = df_sale_raw.merge(df_store_raw, how='left', on='Store')

# 1 - Description data

## 1.1. Rename columns

In [15]:
df1 = df_raw.copy()

In [18]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
            'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
            'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
            'Promo2SinceYear', 'PromoInterval']

#Usando o método underscore do inflection para deixar tudo minúsculo e separado por "_"
snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, cols_old))

#Renomeando as colunas do dataframe
df1.columns = cols_new

## 1.2. Data dimension

In [20]:
print(f'Número de linhas: {df1.shape[0]}')
print(f'Número de colunas: {df1.shape[1]}')

Número de linhas: 1017209
Número de colunas: 18


## 1.3. Data types

In [21]:
df1.dtypes

store                             int64
day_of_week                       int64
date                             object
sales                             int64
customers                         int64
open                              int64
promo                             int64
state_holiday                    object
school_holiday                    int64
store_type                       object
assortment                       object
competition_distance            float64
competition_open_since_month    float64
competition_open_since_year     float64
promo2                            int64
promo2_since_week               float64
promo2_since_year               float64
promo_interval                   object
dtype: object

In [22]:
#Transformando "date" para datetime
df1['date'] = pd.to_datetime(df1['date'])

## 1.4. Check NA

In [23]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5. Fillout NA

In [25]:
df2 = df1.copy()

In [26]:
#competition_distance
df2['competition_distance'] = df2['competition_distance'].apply( lambda x:200000.0 if math.isnan( x ) else x)
        
#cocompetition_open_since_month
df2['competition_open_since_month'] = df2.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_month'] ) else 
                                                           x['competition_open_since_month'], axis=1)
   
#competition_open_since_year
df2['competition_open_since_year'] = df2.apply( lambda x: x['date'].year if math.isnan( x['competition_open_since_year'] ) else 
                                                          x['competition_open_since_year'], axis=1)
     
#promo2_since_week
df2['promo2_since_week'] = df2.apply( lambda x: x['date'].week if math.isnan(x['promo2_since_week'] ) else x['promo2_since_week'], axis=1)
             
#promo2_since_year
df2['promo2_since_year'] = df2.apply( lambda x: x['date'].year if math.isnan(x['promo2_since_year'] ) else x['promo2_since_year'], axis=1)
               
#promo_interval
month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

df2['promo_interval'].fillna(0, inplace=True )
df2['month_map'] = df2['date'].dt.month.map( month_map )
df2['is_promo'] = df2[['promo_interval', 'month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split( ',' ) else 0, axis=1)